In [1]:
# load_sparse.py: modifying for STG
import os
import pickle as pkl
import tensorflow as tf
import h5py
import numpy as np
from time import time
import warnings
from load_sparse import create_network_dat
# from memory_profiler import profile

def sort_indices(indices, *arrays):
    max_ind = np.max(indices) + 1
    if np.iinfo(indices.dtype).max < max_ind * (max_ind + 1) :
        indices = indices.astype(np.int64)
    q = indices[:, 0] * max_ind + indices[:, 1]
    sorted_ind = np.argsort(q)
    sorted_arrays = list(map(lambda arr: arr[sorted_ind], [indices, *arrays]))
    return tuple(sorted_arrays)

# The following function, although it provided a speed up, it creates bunch of tensor copies 
# which are not garbage collected and thus it is not memory efficient
def sort_indices_tf(indices, *arrays):
    indices = tf.cast(indices, dtype=tf.int64)    
    max_ind = tf.reduce_max(indices) + 1
    q = indices[:, 0] * max_ind + indices[:, 1]
    sorted_ind = tf.argsort(q)
    indices = tf.cast(indices, dtype=tf.int32)
    sorted_arrays = [tf.gather(arr, sorted_ind).numpy() for arr in [indices, *arrays]]
    return tuple(sorted_arrays)

2024-12-17 09:07:28.656210: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-17 09:07:28.726426: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-17 09:07:28.726456: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-17 09:07:28.726462: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-17 09:07:28.730939: I tensorflow/core/platform/cpu_feature_g

## Load main network

In [ ]:
path="../GLIF_network/network_dat.pkl"
h5_path="../GLIF_network/network/stg_nodes.h5"
core_only=False
n_neurons=685
seed=3000
connected_selection=False
n_syn_basis=5
tensorflow_speed_up=False


rd = np.random.RandomState(seed=seed)

# Create / Load the network_dat pickle file from the SONATA files
if not os.path.exists(path):
    base_dir = os.path.dirname(path)
    data_dir = os.path.join(base_dir, "network")
    print(f"Creating {path} file...")
    d = create_network_dat(data_dir=data_dir, source='stg', target='stg',
                            output_file=path, save_pkl=True)
    # d = create_network_dat(data_dir='GLIF_network/network', source='v1', target='v1',
    #                         output_file='GLIF_network/network_dat.pkl', save_pkl=True)
else:
    print("Loading network_dat.pkl file...")
    with open(path, "rb") as f:
        d = pkl.load(f)  # d is a dictionary with 'nodes' and 'edges' keys

# Get the number of nodes in the full V1 network
n_nodes = sum([len(a["ids"]) for a in d["nodes"]])  # 296991 total neurons
# Create arrays to convert between bmtk and tf ides
tf_id_to_bmtk_id = np.arange(n_nodes, dtype=np.int32)
bmtk_id_to_tf_id = np.full(n_nodes, -1, dtype=np.int32)

# Extract from the SONATA file the nodes information
# h5_file = h5py.File(h5_path, "r")            
node_data_dict = {}
with h5py.File(h5_path, "r") as h5_file:
    assert np.diff(h5_file["nodes"]["stg"]["node_id"]).var() < 1e-12
    node_type_id = np.array(h5_file['nodes']['stg']['node_type_id'], dtype=np.int32)
            
    # Dynamically get individual node specific parameters:
    # allows for the addition/removal of node parameters during the bmtk build stage
    # without changing this portion of the training code
    node_keys = list(h5_file["nodes"]["stg"]["0"].keys())
    node_group = h5_file["nodes"]["stg"]["0"]
    for key in node_keys:
        if node_group[key].dtype == 'float64' or node_group[key].dtype == 'float32':
            dtype = np.float32
        elif node_group[key].dtype == 'int64' or node_group[key].dtype == 'int32':
            dtype = np.int32
        else:
            dtype = str
        
        data = np.array(node_group[key], dtype=dtype)
        # Clean up key name
        safe_key = ''.join(c if c.isalnum() or c == '_' else '_' for c in key)
        node_data_dict[safe_key] = data
        
r = np.sqrt(node_data_dict['x']**2 + node_data_dict['z']**2)  # the maximum radius is 845

In [ ]:

### CHOOSE THE NETWORK NODES ###
if n_neurons > n_nodes:  # used to be an explicit number: 296991
    raise ValueError(f"There are only {n_nodes} neurons in the network")

elif connected_selection: # this condition takes the n_neurons closest neurons to the origin
    sorted_ind = np.argsort(r)
    sel = np.zeros(n_nodes, dtype=np.bool_)
    sel[sorted_ind[:n_neurons]] = True
    print(f"> Maximum sample radius: {r[sorted_ind[n_neurons - 1]]:.2f}")

elif core_only: # this condition takes the n_neurons closest neurons to the origin (core)
    sel = r < 400
    n_core = np.sum(sel)
    if n_neurons > n_core:
        raise ValueError(f"There are only {n_core} neurons in the network core")
    elif n_neurons > 0 and n_neurons <= n_core:
        (inds,) = np.where(sel)
        take_inds = rd.choice(inds, size=n_neurons, replace=False)
        sel[:] = False
        sel[take_inds] = True

elif n_neurons > 0 and n_neurons <= n_nodes: # this condition randomly selects neurons from the whole STG
    print(n_neurons)
    print(n_nodes)
    legit_neurons = np.arange(n_nodes)
    take_inds = rd.choice(legit_neurons, size=n_neurons, replace=False)
    
    # Check to make sure at least one edge exists between the chosen nodes --> will cause an error later if not
    # Shouldn't be an issue if the network is connected but is a bug for feedforward versions
    edges = d["edges"]
    for edge in edges:
        # Identify which of the 10 types of inputs we have
        edge_type_id = edge["edge_type_id"]
        target_tf_ids = bmtk_id_to_tf_id[edge["target"]]
        source_tf_ids = bmtk_id_to_tf_id[edge["source"]]
        edge_exists = np.logical_and(target_tf_ids != -1, source_tf_ids != -1)
        # find the edges between the selected nodes
        target_tf_ids = target_tf_ids[edge_exists]
        source_tf_ids = source_tf_ids[edge_exists]
    # If no edges exist between the chosen nodes, modify the node selection to include a single edge   
    if target_tf_ids.size == 0 and len(edges) > 0:
        warnings.warn("Due to sparse edges no edges exist between the chosen nodes")
        warnings.warn("Modifying node selection to include a single edge to create a largely feedforward network.")
        # get the source and target nodes IDs of a single edge   
        edge = edges[0]
        target_id = edge["target"][0]
        source_id = edge["source"][0]
        # Check if both source and target nodes are in the selected nodes & replace a selected node if not
        if target_id not in take_inds:            
            take_inds[0] = target_id           
        if source_id not in take_inds:
            take_inds[1] = source_id
            
    # !!! np.empty creates an array with a RANDOM amount of True & False values !!!
    # sel = np.empty(n_nodes, dtype=np.bool_)
    sel = np.zeros(n_nodes, dtype=np.bool_) # Creates an array with ONLY False values
    sel[take_inds] = True # Assigns True to the randomly selected neurons

else: # if no condition is met, all neurons are selected
    sel = np.ones(n_nodes, dtype=np.bool_)

# Get the number of neurons in the chosen network and update the traslation arrays
n_nodes = np.sum(sel)
print(f"> Number of Neurons: {n_nodes}")
tf_id_to_bmtk_id = tf_id_to_bmtk_id[sel] # tf idx '0' corresponds to 'tf_id_to_bmtk_id[0]' bmtk idx
bmtk_id_to_tf_id[tf_id_to_bmtk_id] = np.arange(n_nodes, dtype=np.int32) 
# bmtk idx '0' corresponds to 'bmtk_id_to_tf_id[0]' tf idx which can be '-1' in case
# the bmtk node is not in the tensorflow selection or another value in case it belongs the selection

# for tf_id, bmtk_id in enumerate(tf_id_to_bmtk_id):
#     bmtk_id_to_tf_id[bmtk_id] = tf_id

# Get the properties from the network neurons
#tuning_angle = tuning_angle[sel]
node_type_id = node_type_id[sel]

# Get the target cell type properties from the network neurons    
for key, data in node_data_dict.items():
    node_data_dict[key] = data[sel]




In [ ]:
# GET THE NODES PARAMETERS
n_node_types = len(d["nodes"])
node_params = dict(
    V_th=np.empty(n_node_types, np.float32),
    g=np.empty(n_node_types, np.float32),
    E_L=np.empty(n_node_types, np.float32),
    k=np.empty((n_node_types, 2), np.float32),
    C_m=np.empty(n_node_types, np.float32),
    V_reset=np.empty(n_node_types, np.float32),
    t_ref=np.empty(n_node_types, np.float32),
    asc_amps=np.empty((n_node_types, 2), np.float32),
)

node_type_ids = np.empty(n_nodes, np.int32)
for i, node_type in enumerate(d["nodes"]):
    # get ALL the nodes of the given node type
    tf_ids = bmtk_id_to_tf_id[np.array(node_type["ids"], dtype=np.int32)]
    # choose only those that belong to our model
    tf_ids = tf_ids[tf_ids >= 0]
    # assign them all the same id (which does not relate with the neuron type)
    node_type_ids[tf_ids] = i
    for k, v in node_params.items():
        # save in a dict the information of the nodes
        v[i] = node_type["params"][k]

# GET THE EDGES INFORMATION
t0 = time()
edges = d["edges"]
n_edges = 0
dense_shape = (n_nodes, n_nodes)
indices = []
weights = []
delays = []
syn_ids = []
edge_type_ids = []

for edge in edges:
    # Identify which of the 10 types of inputs we have
    edge_type_id = edge["edge_type_id"]
    target_tf_ids = bmtk_id_to_tf_id[edge["target"]]
    source_tf_ids = bmtk_id_to_tf_id[edge["source"]]
    edge_exists = np.logical_and(target_tf_ids != -1, source_tf_ids != -1)
    # select the edges within our model
    target_tf_ids = target_tf_ids[edge_exists]
    source_tf_ids = source_tf_ids[edge_exists]
    weights_tf = edge["params"]["weight"][edge_exists].astype(np.float32)

    n_new_edge = len(target_tf_ids)
    n_edges += int(n_new_edge)
    
    # all the edges of a given type have the same delay and synaptic id
    delays_tf = np.full(n_new_edge, edge["params"]["delay"], dtype=np.float16)
    syn_id = np.full(n_new_edge, edge["params"]["syn_id"], dtype=np.uint8)

    indices.append(np.array([target_tf_ids, source_tf_ids]).T)
    weights.append(weights_tf)
    delays.append(delays_tf)
    syn_ids.append(syn_id)
    edge_type_ids.append(np.full(n_new_edge, edge_type_id, dtype=np.uint16))

print(f"> Number of Synapses: {n_edges}")
indices = np.concatenate(indices, axis=0, dtype=np.int32)
weights = np.concatenate(weights, axis=0, dtype=np.float32)
delays = np.concatenate(delays, axis=0, dtype=np.float16)
syn_ids = np.concatenate(syn_ids, axis=0, dtype=np.uint8)
edge_type_ids = np.concatenate(edge_type_ids, axis=0, dtype=np.uint16)

# sort indices by considering first all the targets of node 0, then all of node 1, ...
# indices, weights, delays, tau_syn_weights_array, syn_ids = sort_indices(indices, weights, delays, tau_syn_weights_array, syn_ids)
# indices, weights, delays, syn_ids = sort_indices_tf(indices, weights, delays, syn_ids)
# indices, weights, delays, syn_ids = sort_indices_tf(indices, weights, delays, syn_ids)

if tensorflow_speed_up:
    indices, weights, delays, syn_ids, edge_type_ids = sort_indices_tf(indices, weights, delays, syn_ids, edge_type_ids)
else:
    indices, weights, delays, syn_ids, edge_type_ids = sort_indices(indices, weights, delays, syn_ids, edge_type_ids)

network = dict(
    #tuning_angle=tuning_angle,
    node_type_id=node_type_id,
    n_nodes=n_nodes,
    n_edges=n_edges,
    node_params=node_params,
    node_type_ids=node_type_ids,
    synapses=dict(indices=indices, weights=weights, delays=delays, 
                    syn_ids=syn_ids, edge_type_ids=edge_type_ids,
                    dense_shape=dense_shape),
    tf_id_to_bmtk_id=tf_id_to_bmtk_id,
    bmtk_id_to_tf_id=bmtk_id_to_tf_id,
)

# Add the node data to the network dictionary
network.update(node_data_dict)

## Load input networks

In [2]:
import load_sparse

network = load_sparse.load_network(
                    path="../GLIF_network/network_dat.pkl",
                    h5_path="../GLIF_network/network/stg_nodes.h5",
                    core_only=False,
                    n_neurons=685,
                    seed=3000,
                    connected_selection=False,
                    n_syn_basis=5,
                    tensorflow_speed_up=False
                    )

data_dir="../GLIF_network/network"
lgn_path="../GLIF_network/aud_input_dat.pkl"
bkg_path="../GLIF_network/bkg_input_dat.pkl"

Loading network_dat.pkl file...
Number nodes available:  685
Number neurons to select:  685
> Number of Neurons: 685
> Number of Synapses: 1


/allen/programs/mindscope/workgroups/realistic-model/elena.westeinde/human_model/STG_training/v1_model_utils/load_sparse.py:277: UserWarning: Due to sparsity of edges no edges exist between the chosen nodes
  warnings.warn("Due to sparsity of edges no edges exist between the chosen nodes")
/allen/programs/mindscope/workgroups/realistic-model/elena.westeinde/human_model/STG_training/v1_model_utils/load_sparse.py:278: UserWarning: Modifying node selection to include a single edge to create a largely feedforward network.
  warnings.warn("Modifying node selection to include a single edge to create a largely feedforward network.")


In [3]:
dt=1
bmtk_id_to_tf_id=network["bmtk_id_to_tf_id"] 
tensorflow_speed_up=False

lgn_path = os.path.join(data_dir, "aud_input_dat.pkl")
bkg_path = os.path.join(data_dir, "bkg_input_dat.pkl")
# LOAD THE LGN INPUT
if not os.path.exists(lgn_path):
    # print("Creating lgn_input_dat.pkl file...")
    print(f"Creating {lgn_path} file...")
    # Process LGN input network
    lgn_input = create_network_dat(data_dir=data_dir, source='aud_input', target='stg',
                                    output_file=lgn_path, save_pkl=True)
    # lgn_input = create_network_dat(data_dir='GLIF_network/network', source='lgn', target='v1', 
    #                                 output_file=lgn_path, save_pkl=True)
    print("Done.")
else:
    with open(lgn_path, "rb") as f:
        lgn_input = pkl.load(f)
        
# create_network_dat specifically does not load node info for the LGN and BKG inputs --> because the nodes have no trainable parameters?
    

In [9]:
# LOAD THE BACKGROUND INPUT
if not os.path.exists(bkg_path):
    # print("Creating bkg_input_dat.pkl file...")
    print(f"Creating {bkg_path} file...")
    # Process LGN input network
    bkg_input = create_network_dat(data_dir=data_dir, source='bkg', target='stg',
                                    output_file=bkg_path, save_pkl=True)
    # bkg_input = create_network_dat(data_dir='GLIF_network/network', source='bkg', target='v1', 
    #                                 output_file=bkg_path, save_pkl=True)
    print("Done.")
else:
    with open(bkg_path, "rb") as f:
        bkg_input = pkl.load(f)

In [32]:
input_edges = [lgn_input['edges'], bkg_input['edges']]
for edge in lgn_input['edges']:
    target_bmtk_id = edge["target"]
    print(max(target_bmtk_id))
    print(len(bmtk_id_to_tf_id))
    if max(target_bmtk_id) >= len(bmtk_id_to_tf_id):
        print('true')
        keep_idx = target_bmtk_id < len(bmtk_id_to_tf_id)
        target_bmtk_id = target_bmtk_id[keep_idx]
        source_tf_id = edge["source"]
        weights_tf = edge["params"]["weight"]
#         print("Warning: Input target nodes exceed the number of nodes in the network, reducing target node IDs")

# print(bkg_input['edges'])
# print(bkg_input['edges'][0]["target"])
# print(bkg_input['edges'][0]["source"])

print(source_tf_id)

4451
685
true
[ 557 2113 4077 ... 2952  239 3261]


In [33]:
# Unite the edges information of the LGN and the background noise
input_edges = [lgn_input['edges'], bkg_input['edges']]
    
input_populations = []
for idx, input_population in enumerate(input_edges):
    post_indices = []
    pre_indices = []
    weights = []
    delays = []
    syn_ids = []

    for edge in input_population:  # input_population[1]:
        # Identify the which of the 10 types of inputs we have
        # r = edge["params"]["receptor_type"] - 1
        # r takes values within 0 - 9
        target_bmtk_id = edge["target"]
        source_tf_id = edge["source"]
        weights_tf = edge["params"]["weight"]
        
        if max(target_bmtk_id) >= len(bmtk_id_to_tf_id):
            keep_idx = target_bmtk_id < len(bmtk_id_to_tf_id)
            target_bmtk_id = target_bmtk_id[keep_idx]
            source_tf_id = source_tf_id[keep_idx]
            weights_tf = weights_tf[keep_idx]
            print("Warning: Input target nodes exceed the number of nodes in the tf network, reducing input node IDs")
        
        # delays_tf = np.array(edge["params"]["delay"], dtype=np.float16)
        # syn_id = np.array(edge["params"]["syn_id"], dtype=np.uint8)
        # delays_tf = (np.zeros_like(weights_tf) + edge["params"]["delay"]).astype(np.float16)
        # syn_id = (np.zeros_like(weights_tf) + edge["params"]["syn_id"]).astype(np.uint8)

        n_new_edge = len(target_bmtk_id)
        delays_tf = np.full(n_new_edge, edge["params"]["delay"], dtype=np.float16)
        syn_id = np.full(n_new_edge, edge["params"]["syn_id"], dtype=np.uint8)

        if bmtk_id_to_tf_id is not None:
            # check if the given edges exist in our model
            # (notice that only the target must exist since the source is within the LGN module)
            # This means that source index is within 0-17400
            target_tf_id = bmtk_id_to_tf_id[target_bmtk_id]
            edge_exists = target_tf_id != -1
            target_tf_id = target_tf_id[edge_exists]
            source_tf_id = source_tf_id[edge_exists]
            weights_tf = weights_tf[edge_exists]
            delays_tf = delays_tf[edge_exists]
            syn_id = syn_id[edge_exists]

            # we multiply by 10 the indices and add r to identify the receptor_type easily:
            # if target id is divisible by 10 the receptor_type is 0,
            # if it is rest is 1 by dividing by 10 then its receptor type is 1, and so on...
            # extend acts by extending the list with the given object
            # new_target_tf_id = target_tf_id * max_n_receptors + r
            new_target_tf_id = target_tf_id
            post_indices.append(new_target_tf_id)
            pre_indices.append(source_tf_id)
            weights.append(weights_tf)
            delays.append(delays_tf)
            syn_ids.append(syn_id)

    post_indices = np.concatenate(post_indices, axis=0, dtype=np.int32)
    pre_indices = np.concatenate(pre_indices, axis=0, dtype=np.int32)
    weights = np.concatenate(weights, axis=0, dtype=np.float32)
    delays = np.concatenate(delays, axis=0, dtype=np.float16)
    syn_ids = np.concatenate(syn_ids, axis=0, dtype=np.uint8)

    # first column are the post indices and second column the pre indices
    indices = np.stack([post_indices, pre_indices], axis=-1)

    # Sort indices
    # indices, weights, delays, syn_ids = sort_indices_tf(indices, weights, delays, syn_ids)
    if tensorflow_speed_up:
        indices, weights, delays, syn_ids = sort_indices_tf(indices, weights, delays, syn_ids)
    else:
        indices, weights, delays, syn_ids = sort_indices(indices, weights, delays, syn_ids)

    if idx == 0:
        # we load the LGN nodes and their positions
        lgn_nodes_h5_file = h5py.File(os.path.join(data_dir,"aud_input_nodes.h5"), "r")
        #lgn_nodes_h5_file = h5py.File(f"{data_dir}/network/lgn_nodes.h5", "r")
        n_inputs = len(lgn_nodes_h5_file["nodes"]["aud_input"]["node_id"])
    else:
        # we load the background nodes and their positions
        bkg_nodes_h5_file = h5py.File(os.path.join(data_dir,"bkg_nodes.h5"), "r")
        #bkg_nodes_h5_file = h5py.File(f"{data_dir}/network/bkg_nodes.h5", "r")
        n_inputs = len(bkg_nodes_h5_file["nodes"]["bkg"]["node_id"])

    input_populations.append(
        dict(
            n_inputs=n_inputs,
            indices=indices,
            weights=weights,
            delays=delays,
            syn_ids=syn_ids,
        )
    )


In [3]:
data_dir="../GLIF_network/network"
lgn_path="../GLIF_network/aud_input_dat.pkl"
bkg_path="../GLIF_network/bkg_input_dat.pkl"
dt=1
bmtk_id_to_tf_id=network["bmtk_id_to_tf_id"] 
tensorflow_speed_up=False

inputs = load_sparse.load_input(
                    start=1000, 
                    duration=1000,
                    dt=1,
                    data_dir="../GLIF_network/network",
                    lgn_path="../GLIF_network/aud_input_dat.pkl",
                    bkg_path="../GLIF_network/bkg_input_dat.pkl",
                    bmtk_id_to_tf_id=network["bmtk_id_to_tf_id"], 
                    tensorflow_speed_up=False
)
    
    
lgn_input = inputs[0]
bkg_input = inputs[1]

In [4]:
def reduce_input_population(input_population, new_n_input, seed=3000):
    rd = np.random.RandomState(seed=seed)

    in_ind = input_population["indices"]
    in_weights = input_population["weights"]
    in_delays = input_population["delays"]
    in_syn_ids = input_population["syn_ids"]

    # we take input_population['n_inputs'] neurons from a list of new_n_input with replace,
    # which means that in the end there can be less than new_n_input neurons of the LGN,
    # but they are randonmly selected
    assignment = rd.choice(np.arange(new_n_input, dtype=np.int32), size=input_population["n_inputs"], replace=True)

    # Create a tuple of indices for vectorized operations
    post_indices = in_ind[:, 0]
    input_neuron_indices = in_ind[:, 1]
    assigned_neuron_indices = assignment[input_neuron_indices]

    # Calculate unique pairs and their indices in the original array
    unique_pairs, inverse_indices = np.unique(np.stack((post_indices, assigned_neuron_indices), axis=1), axis=0, return_inverse=True)

    # Accumulate weights for repeated pairs
    new_in_weights = np.bincount(inverse_indices, weights=in_weights)

    # Lets get the average delay by the connection strength (synaptic weight), 
    # assuming stronger connections might have a more significant impact on the timing.
    # Calculate mean delays for each unique pair
    # First, accumulate the total delays for each unique pair
    total_delays = np.bincount(inverse_indices, weights=in_delays)
    # Count the occurrences of each unique pair to divide and get the mean
    counts = np.bincount(inverse_indices)
    # Calculate the mean by dividing total delays by counts
    new_in_delays = total_delays / counts

    # Similarly for the syn_ids
    new_in_syn_ids = np.bincount(inverse_indices, weights=in_syn_ids.astype(np.int32))
    new_in_syn_ids = (new_in_syn_ids/counts).astype(np.uint8)

    # new_in_ind, new_in_weights, new_in_delays = sort_input_indices(
    #     new_in_ind, new_in_weights, new_in_delays
    # )

    new_in_ind, new_in_weights, new_in_delays, new_in_syn_ids = sort_indices(
        unique_pairs, new_in_weights, new_in_delays, new_in_syn_ids
    )
    
    new_input_population = dict(
        n_inputs=new_n_input,
        indices=new_in_ind.astype(np.int32),
        weights=new_in_weights.astype(np.float32),
        delays=new_in_delays.astype(np.float16),
        syn_ids=new_in_syn_ids,
        # spikes=None,
    )

    return new_input_population

In [5]:
lgn_input

{'n_inputs': 500,
 'indices': array([[  0,   7],
        [  0,  10],
        [  0,  24],
        ...,
        [684, 495],
        [684, 496],
        [684, 498]], dtype=int32),
 'weights': array([0.05, 0.05, 0.05, ..., 0.05, 0.05, 0.05], dtype=float32),
 'delays': array([1.7, 1.7, 1.7, ..., 1.7, 1.7, 1.7], dtype=float16),
 'syn_ids': array([62, 62, 62, ..., 62, 62, 62], dtype=uint8)}

In [6]:
# If required reduce the number of LGN inputs
n_input = 500
fseed = 3000
if n_input != 17400:
    lgn_input = reduce_input_population(lgn_input, n_input, seed=fseed)

In [7]:
lgn_input

{'n_inputs': 500,
 'indices': array([[  0,   0],
        [  0,  16],
        [  0,  18],
        ...,
        [684, 481],
        [684, 491],
        [684, 493]], dtype=int32),
 'weights': array([0.05, 0.05, 0.1 , ..., 0.05, 0.05, 0.1 ], dtype=float32),
 'delays': array([1.7, 1.7, 1.7, ..., 1.7, 1.7, 1.7], dtype=float16),
 'syn_ids': array([62, 62, 62, ..., 62, 62, 62], dtype=uint8)}